In [1]:
import pandas as pd

data = pd.read_csv('../Datasets/data_with_embeddings.csv')

In [2]:
data.columns

Index(['num_journeys', 'max_journey', 'discover', 'number_accounts',
       'one_more_journey', 'most_repeated_event', 'average_length_seq',
       'approved_credit', 'order_ships', 'has_prospecting',
       'has_pre_application', 'initial_device', 'time_in_discover',
       'time_in_apply', 'event_id_0', 'event_id_1', 'event_id_2', 'event_id_3',
       'event_id_4', 'time_0', 'time_1', 'time_2', 'time_3', 'time_4'],
      dtype='object')

In [3]:
data.head()

,num_journeys,max_journey,discover,number_accounts,one_more_journey,most_repeated_event,average_length_seq,approved_credit,order_ships,has_prospecting,...,event_id_0,event_id_1,event_id_2,event_id_3,event_id_4,time_0,time_1,time_2,time_3,time_4
0,1.0,27.0,1.0,1.0,0.0,4.0,24.0,1.0,0.0,1.0,...,1.379312,-0.104741,0.578822,0.915176,0.488270,-131084.89000,-18791.02100,-138090.92000,-161141.1400,-53334.4180
1,1.0,5.0,1.0,1.0,0.0,2.0,3.0,1.0,0.0,1.0,...,0.664583,0.928158,-0.229372,1.239874,1.322270,888.64374,1697.03310,1182.47230,3313.5122,2466.7332
2,1.0,30.0,1.0,1.0,0.0,5.0,30.0,1.0,1.0,0.0,...,1.073141,-1.276163,-0.071933,0.272088,0.046549,887.77325,1239.86610,-306.40286,1656.2661,1766.3341
3,1.0,7.0,1.0,1.0,0.0,4.0,7.0,1.0,0.0,0.0,...,2.109451,1.346191,0.803371,2.301213,1.810838,20723.34200,28942.26200,-7152.37900,38662.3120,41231.6330
4,1.0,22.0,1.0,1.0,0.0,19.0,17.0,1.0,0.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,224.84004,421.51904,345.40106,854.2560,612.7782


In [4]:
from sklearn.model_selection import train_test_split

X = data.drop(['order_ships'], axis=1)
y = data['order_ships']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np

class CustomerJourneyDataset(Dataset):
    def __init__(self, X, y):
        # Convert DataFrame to NumPy array if it's not already an array
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values

        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)  # Ensure y is 2D for BCEWithLogitsLoss
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# Assuming X_train, y_train are your features and labels
train_dataset = CustomerJourneyDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [14]:
# Make sure GPU is loaded for training
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: NVIDIA GeForce RTX 3060


In [27]:
import torch.nn as nn

class FCNNModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(FCNNModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, output_dim)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Adjust model instantiation for FCNN
model = FCNNModel(input_dim=23, output_dim=1)


In [30]:
sequences, labels = next(iter(train_loader))
#sequences, labels = sequences.to(device), labels.to(device)
print(model(sequences))

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], grad_fn=<AddmmBackward0>)


In [29]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 100

for epoch in range(num_epochs):
    for sequences, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(sequences)
        #print(outputs)
        loss = criterion(outputs.squeeze(), labels.squeeze())
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: nan
Epoch 2, Loss: nan
Epoch 3, Loss: nan
Epoch 4, Loss: nan
Epoch 5, Loss: nan


KeyboardInterrupt: 